# FedAvg Experiment

In [1]:
#@test {"skip": true}

!which python
import nest_asyncio
nest_asyncio.apply()

%load_ext tensorboard

from matplotlib import pyplot as plt
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

/usr/local/bin/python


In [2]:

import collections

from IPython.display import display, HTML, IFrame

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

print(tff.__version__)

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

np.random.seed(0)
#def greetings():
#  display(HTML('<b><font size="6" color="#ff00f4">Greetings, virtual tutorial participants!</font></b>'))
#  return True
#l = tff.federated_computation(greetings)()


0.18.0


In [3]:
# Code for loading federated data from TFF repository
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data(only_digits=False)

In [33]:
len(emnist_train.client_ids), len(emnist_test.client_ids)

(3400, 3400)

In [4]:
# Let's look at the shape of our data
example_dataset = emnist_train.create_tf_dataset_for_client(
    emnist_train.client_ids[0])
#example_dataset
example_dataset.element_spec

OrderedDict([('label', TensorSpec(shape=(), dtype=tf.int32, name=None)),
             ('pixels',
              TensorSpec(shape=(28, 28), dtype=tf.float32, name=None))])

In [5]:
# Let's select an example dataset from one of our simulated clients
example_dataset = emnist_train.create_tf_dataset_for_client(
    emnist_train.client_ids[45])

# Your code to get an example element from one client:
example_element = next(iter(example_dataset))

example_element['label'].numpy()

55

In [ ]:
plt.imshow(example_element['pixels'].numpy(), cmap='gray', aspect='equal')
plt.grid(False)
_ = plt.show()

**Exploring non-iid data**

In [ ]:
## Example MNIST digits for one client
f = plt.figure(figsize=(20,4))
j = 0

for e in example_dataset.take(40):
  plt.subplot(4, 10, j+1)
  plt.imshow(e['pixels'].numpy(), cmap='gray', aspect='equal')
  plt.axis('off')
  j += 1

In [ ]:
# Number of examples per layer for a sample of clients
f = plt.figure(figsize=(12,7))
f.suptitle("Label Counts for a Sample of Clients")
for i in range(6):
  ds = emnist_train.create_tf_dataset_for_client(emnist_train.client_ids[i])
  k = collections.defaultdict(list)
  for e in ds:
    k[e['label'].numpy()].append(e['label'].numpy())
  plt.subplot(2, 3, i+1)
  plt.title("Client {}".format(i))
  for j in range(62):
    plt.hist(k[j], density=False, bins=[i for i in range(62)])

In [ ]:
for i in range(2,5):
  ds = emnist_train.create_tf_dataset_for_client(emnist_train.client_ids[i])
  k = collections.defaultdict(list)
  for e in ds:
    k[e['label'].numpy()].append(e['pixels'].numpy())
  f = plt.figure(i, figsize=(12,10))
  f.suptitle("Client #{}'s Mean Image Per Label".format(i))
  for j in range(20):
    mn_img = np.mean(k[j],0)
    plt.subplot(2, 10, j+1)
    if (mn_img.size==1):
      continue
    plt.imshow(mn_img.reshape((28,28)))#,cmap='gray') 
    plt.axis('off')

### Preprocessing the data


In [6]:
NUM_CLIENTS = 20
NUM_EPOCHS = 1
BATCH_SIZE = 20
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER=10

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
    return collections.OrderedDict(
        x=tf.reshape(element['pixels'], [-1, 28,28,1]),
        y=tf.reshape(element['label'], [-1, 1]))

  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
      BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

In [7]:
preprocessed_example_dataset = preprocess(example_dataset)

sample_batch = tf.nest.map_structure(lambda x: x.numpy(),
                                     next(iter(preprocessed_example_dataset)))


In [8]:
def make_federated_data(client_data, client_ids):
  return [
      preprocess(client_data.create_tf_dataset_for_client(x))
      for x in client_ids
  ]

In [9]:
import random
shuffled_ids = emnist_train.client_ids.copy()
random.shuffle(shuffled_ids)
shuffled_ids_train = shuffled_ids[0:2500]


In [10]:
def create_keras_model():
    
  data_format = 'channels_last'
  initializer = tf.keras.initializers.RandomNormal(seed=0)
  return tf.keras.models.Sequential([
      tf.keras.layers.Input(shape=(28, 28,1)),
      tf.keras.layers.Conv2D(32,(3,3), activation='relu'),
      tf.keras.layers.Conv2D(64,(3,3), activation='relu'),
      tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)),
      tf.keras.layers.Dropout(rate=0.75),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu', kernel_initializer=initializer),
      tf.keras.layers.Dropout(rate=0.5, seed=1),
      tf.keras.layers.Dense(62, kernel_initializer=initializer),
      tf.keras.layers.Softmax()
  ])

## Centralized training 

In [ ]:
## Centralized training with keras ---------------------------------------------

# This is separate from the TFF tutorial, and demonstrates how to train a
# Keras model in a centralized fashion (contrasting training in a federated env)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocess the data (these are NumPy arrays)
x_train = x_train.reshape(60000, 28,28,1).astype("float32") / 255

y_train = y_train.astype("float32")

mod = create_keras_model()
mod.compile(
    optimizer=tf.keras.optimizers.RMSprop(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)
h = mod.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=2
)

# ------------------------------------------------------------------------------

In [11]:
def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=preprocessed_example_dataset.element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [12]:
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.05),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0, momentum=0.9))

In [13]:
state = iterative_process.initialize()

In [14]:
NUM_ROUNDS = 1000
import time
start = time.process_time()

weights = None 
for round_num in range(0, NUM_ROUNDS):
    sample_clients = np.random.choice(shuffled_ids_train, NUM_CLIENTS, replace=False)
    federated_train_data = make_federated_data(emnist_train, sample_clients)
    state, metrics = iterative_process.next(state, federated_train_data)
    print('round {:2d}, metrics={}'.format(round_num, metrics['train']))
    weights = state.model.trainable

print(time.process_time() - start)



round  0, metrics=OrderedDict([('sparse_categorical_accuracy', 0.030295802), ('loss', 4.134774)])
round  1, metrics=OrderedDict([('sparse_categorical_accuracy', 0.041244082), ('loss', 4.011042)])
round  2, metrics=OrderedDict([('sparse_categorical_accuracy', 0.04234694), ('loss', 3.887426)])
round  3, metrics=OrderedDict([('sparse_categorical_accuracy', 0.05119108), ('loss', 3.7812872)])
round  4, metrics=OrderedDict([('sparse_categorical_accuracy', 0.055402298), ('loss', 3.7561102)])
round  5, metrics=OrderedDict([('sparse_categorical_accuracy', 0.051530212), ('loss', 3.729875)])
round  6, metrics=OrderedDict([('sparse_categorical_accuracy', 0.05535665), ('loss', 3.7037838)])
round  7, metrics=OrderedDict([('sparse_categorical_accuracy', 0.056403268), ('loss', 3.6642566)])
round  8, metrics=OrderedDict([('sparse_categorical_accuracy', 0.047147002), ('loss', 3.7189343)])
round  9, metrics=OrderedDict([('sparse_categorical_accuracy', 0.06214848), ('loss', 3.641449)])
round 10, metrics=O

FileNotFoundError: [Errno 2] No such file or directory: 'global_model.pkl'

In [15]:

import pickle
with open('global_model.pkl', 'wb') as f:
    pickle.dump(weights,f)
    f.close()

In [ ]:
#@test {"skip": true}
import os
import shutil

logdir = "/tmp/logs/scalars/training/"
if os.path.exists(logdir):
  shutil.rmtree(logdir)

# Your code to create a summary writer:
summary_writer = tf.summary.create_file_writer(logdir)

state = iterative_process.initialize()

In [ ]:
#@test {"skip": true}
with summary_writer.as_default():
  for round_num in range(1, NUM_ROUNDS):
    state, metrics = iterative_process.next(state, federated_train_data)
    print('round {:2d}, metrics={}'.format(round_num, metrics['train']))
    for name, value in metrics['train'].items():
      tf.summary.scalar(name, value, step=round_num)

In [ ]:
#@test {"skip": true}
%tensorboard --logdir /tmp/logs/scalars/ --port=0

# Evaluation

In [16]:
# Construct federated evaluation computation here:
evaluation = tff.learning.build_federated_evaluation(model_fn)

In [17]:
import random
shuffled_ids = emnist_test.client_ids.copy()
random.shuffle(shuffled_ids)
sample_clients = shuffled_ids_train[0:2500]

federated_test_data = make_federated_data(emnist_test, sample_clients)

len(federated_test_data), federated_test_data[0]

(2500,
 <PrefetchDataset shapes: OrderedDict([(x, (None, 28, 28, 1)), (y, (None, 1))]), types: OrderedDict([(x, tf.float32), (y, tf.int32)])>)

In [18]:
# Run evaluation on the test data here, using the federated model produced from 
# training:
test_metrics = evaluation(state.model, federated_test_data)

In [19]:
str(test_metrics)

"OrderedDict([('sparse_categorical_accuracy', 0.8356301), ('loss', 0.48737013)])"

In [ ]:
ckpt_manager = tff.simulation.FileCheckpointManager(root_dir="/src/training/")
ckpt_manager.save_checkpoint(tff.learning.framework.ServerState(state.model,state,None,None), round_num=1000)